In [57]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 11 11:15:49 2019

@author: nasir

Implements
Amplitude Selective Filtering preprocessing step for POS.

"""

import numpy as np
from numpy.linalg import inv
import math
# import scipy.io
# mat = scipy.io.loadmat('c.mat')

def ASF(C):
    alpha=.002;delta=0.0001
    C_=np.dot(inv(np.diag(np.mean(C,1))),C)-1
    L = C.shape[1]
    F=np.fft.fft(C_)/L
    W=delta/(1e-12+np.abs(F[0,:]))
    W=W.astype(np.complex)

    W[np.abs(F[0,:]) < alpha]= 1

    W=np.stack((W,W,W),axis=0)
    F_=F*W

    C__ = np.dot(np.diag(np.mean(C,1)) ,(np.fft.ifft(F_)+1))

    C__ = C__.astype(np.float)
    return C__

# C = mat['C']
# A = ASF(C)

In [56]:
# -*- coding: utf-8 -*-
"""
Implements
Color distortion Filtering preprocessing step for POS.
"""

import numpy as np
from numpy.linalg import inv
import math
#import scipy.io
#mat = scipy.io.loadmat('c.mat')

def CDF(C, B):
    C_ = np.matmul(inv(np.diag(np.mean(C,1))),C)-1
    F = np.fft.fft(C_)
    S = np.dot(np.expand_dims(np.array([-1/math.sqrt(6), 2/math.sqrt(6), -1/math.sqrt(6)]), axis=0), F)
    W = np.real((S* S.conj()) / np.sum((F * F.conj()),0)[None,:])
    W[:, 0:B[0]] = 0
    W[:, B[1]+1:] = 0

    F_ = F * W
    iF = (np.fft.ifft(F_) + 1).real
    C__ = np.matmul(np.diag(np.mean(C,1)) , iF)
    C = C__.astype(np.float)
    return C

#C = mat['C']
#CDF(C)

In [58]:

import torch
from torch.autograd import Variable
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True
# from unet_models import UNet16, unet11
class FaceSegGPU:
    def __init__(self, bs, size=256):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.net = unet11('unet_celeba.pth', pretrained=True).to(self.device)

        # self.net = UNet16(pretrained=True).to(self.device)
        # self.net.load_state_dict(torch.load('unet16.pth'))
        self.net.eval()
        sample = Variable(torch.rand(bs,3,size,size).to(self.device))
        self.net(sample)
        #  = torch.jit.trace(self.net, sample)
        print('___init___')

    def get_mask(self, images, shape):
        # images = Variable(torch.tensor(images, dtype=torch.float,requires_grad=False).to(device=self.device))
        pred = self.net(images)
        pred= torch.nn.functional.interpolate(pred, size=[shape[1], shape[2]])
        pred = pred.squeeze()
        mask = (pred > 0.8)
        segmentation = mask.cpu().numpy()
        return segmentation.astype('float')

    def apply_masks(self, frames_transformed, frames):
        masks = self.get_mask(frames_transformed, frames.shape)
        frames[masks==0] = 0.0
        return frames


In [59]:
from torch import nn
import torch
from torchvision import models
import torchvision
from torch.nn import functional as F


def conv3x3(in_, out):
    return nn.Conv2d(in_, out, 3, padding=1)


class ConvRelu(nn.Module):
    def __init__(self, in_: int, out: int):
        super(ConvRelu, self).__init__()
        self.conv = conv3x3(in_, out)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        return x


class DecoderBlock(nn.Module):
    """
    Paramaters for Deconvolution were chosen to avoid artifacts, following
    link https://distill.pub/2016/deconv-checkerboard/
    """

    def __init__(self, in_channels, middle_channels, out_channels, is_deconv=True):
        super(DecoderBlock, self).__init__()
        self.in_channels = in_channels

        if is_deconv:
            self.block = nn.Sequential(
                ConvRelu(in_channels, middle_channels),
                nn.ConvTranspose2d(middle_channels, out_channels, kernel_size=4, stride=2,
                                   padding=1),
                nn.ReLU(inplace=True)
            )
        else:
            self.block = nn.Sequential(
                nn.Upsample(scale_factor=2, mode='bilinear'),
                ConvRelu(in_channels, middle_channels),
                ConvRelu(middle_channels, out_channels),
            )

    def forward(self, x):
        return self.block(x)


class UNet11(nn.Module):
    def __init__(self, num_classes=1, num_filters=32, pretrained=False):
        """
        :param num_classes:
        :param num_filters:
        :param pretrained:
            False - no pre-trained network used
            True - encoder pre-trained with VGG11
        """
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)

        self.num_classes = num_classes

        self.encoder = models.vgg11(pretrained=pretrained).features

        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Sequential(self.encoder[0],
                                   self.relu)

        self.conv2 = nn.Sequential(self.encoder[3],
                                   self.relu)

        self.conv3 = nn.Sequential(
            self.encoder[6],
            self.relu,
            self.encoder[8],
            self.relu,
        )
        self.conv4 = nn.Sequential(
            self.encoder[11],
            self.relu,
            self.encoder[13],
            self.relu,
        )

        self.conv5 = nn.Sequential(
            self.encoder[16],
            self.relu,
            self.encoder[18],
            self.relu,
        )

        self.center = DecoderBlock(256 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv=True)
        self.dec5 = DecoderBlock(512 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv=True)
        self.dec4 = DecoderBlock(512 + num_filters * 8, num_filters * 8 * 2, num_filters * 4, is_deconv=True)
        self.dec3 = DecoderBlock(256 + num_filters * 4, num_filters * 4 * 2, num_filters * 2, is_deconv=True)
        self.dec2 = DecoderBlock(128 + num_filters * 2, num_filters * 2 * 2, num_filters, is_deconv=True)
        self.dec1 = ConvRelu(64 + num_filters, num_filters)

        self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv2 = self.conv2(self.pool(conv1))
        conv3 = self.conv3(self.pool(conv2))
        conv4 = self.conv4(self.pool(conv3))
        conv5 = self.conv5(self.pool(conv4))
        center = self.center(self.pool(conv5))

        dec5 = self.dec5(torch.cat([center, conv5], 1))
        dec4 = self.dec4(torch.cat([dec5, conv4], 1))
        dec3 = self.dec3(torch.cat([dec4, conv3], 1))
        dec2 = self.dec2(torch.cat([dec3, conv2], 1))
        dec1 = self.dec1(torch.cat([dec2, conv1], 1))

        if self.num_classes > 1:
            x_out = F.log_softmax(self.final(dec1), dim=1)
        else:
            x_out = self.final(dec1)

        return x_out


class UNet16(nn.Module):
    def __init__(self, num_classes=1, num_filters=32, pretrained=False):
        """
        :param num_classes:
        :param num_filters:
        :param pretrained:
            False - no pre-trained network used
            True - encoder pre-trained with VGG11
        """
        super().__init__()
        self.num_classes = num_classes

        self.pool = nn.MaxPool2d(2, 2)

        self.encoder = torchvision.models.vgg16(pretrained=pretrained).features

        self.relu = nn.ReLU(inplace=True)

        self.conv1 = nn.Sequential(self.encoder[0],
                                   self.relu,
                                   self.encoder[2],
                                   self.relu)

        self.conv2 = nn.Sequential(self.encoder[5],
                                   self.relu,
                                   self.encoder[7],
                                   self.relu)

        self.conv3 = nn.Sequential(self.encoder[10],
                                   self.relu,
                                   self.encoder[12],
                                   self.relu,
                                   self.encoder[14],
                                   self.relu)

        self.conv4 = nn.Sequential(self.encoder[17],
                                   self.relu,
                                   self.encoder[19],
                                   self.relu,
                                   self.encoder[21],
                                   self.relu)

        self.conv5 = nn.Sequential(self.encoder[24],
                                   self.relu,
                                   self.encoder[26],
                                   self.relu,
                                   self.encoder[28],
                                   self.relu)

        self.center = DecoderBlock(512, num_filters * 8 * 2, num_filters * 8)

        self.dec5 = DecoderBlock(512 + num_filters * 8, num_filters * 8 * 2, num_filters * 8)
        self.dec4 = DecoderBlock(512 + num_filters * 8, num_filters * 8 * 2, num_filters * 8)
        self.dec3 = DecoderBlock(256 + num_filters * 8, num_filters * 4 * 2, num_filters * 2)
        self.dec2 = DecoderBlock(128 + num_filters * 2, num_filters * 2 * 2, num_filters)
        self.dec1 = ConvRelu(64 + num_filters, num_filters)
        self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv2 = self.conv2(self.pool(conv1))
        conv3 = self.conv3(self.pool(conv2))
        conv4 = self.conv4(self.pool(conv3))
        conv5 = self.conv5(self.pool(conv4))

        center = self.center(self.pool(conv5))

        dec5 = self.dec5(torch.cat([center, conv5], 1))

        dec4 = self.dec4(torch.cat([dec5, conv4], 1))
        dec3 = self.dec3(torch.cat([dec4, conv3], 1))
        dec2 = self.dec2(torch.cat([dec3, conv2], 1))
        dec1 = self.dec1(torch.cat([dec2, conv1], 1))

        if self.num_classes > 1:
            x_out = F.log_softmax(self.final(dec1), dim=1)
        else:
            x_out = self.final(dec1)

        return x_out


class DecoderBlockLinkNet(nn.Module):
    def __init__(self, in_channels, n_filters):
        super().__init__()

        self.relu = nn.ReLU(inplace=True)

        # B, C, H, W -> B, C/4, H, W
        self.conv1 = nn.Conv2d(in_channels, in_channels // 4, 1)
        self.norm1 = nn.BatchNorm2d(in_channels // 4)

        # B, C/4, H, W -> B, C/4, 2 * H, 2 * W
        self.deconv2 = nn.ConvTranspose2d(in_channels // 4, in_channels // 4, kernel_size=4,
                                          stride=2, padding=1, output_padding=0)
        self.norm2 = nn.BatchNorm2d(in_channels // 4)

        # B, C/4, H, W -> B, C, H, W
        self.conv3 = nn.Conv2d(in_channels // 4, n_filters, 1)
        self.norm3 = nn.BatchNorm2d(n_filters)

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu(x)
        x = self.deconv2(x)
        x = self.norm2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu(x)
        return x


class LinkNet34(nn.Module):
    def __init__(self, num_classes=1, num_channels=3, pretrained=True):
        super().__init__()
        assert num_channels == 3
        self.num_classes = num_classes
        filters = [64, 128, 256, 512]
        resnet = models.resnet34(pretrained=pretrained)

        self.firstconv = resnet.conv1
        self.firstbn = resnet.bn1
        self.firstrelu = resnet.relu
        self.firstmaxpool = resnet.maxpool
        self.encoder1 = resnet.layer1
        self.encoder2 = resnet.layer2
        self.encoder3 = resnet.layer3
        self.encoder4 = resnet.layer4

        # Decoder
        self.decoder4 = DecoderBlockLinkNet(filters[3], filters[2])
        self.decoder3 = DecoderBlockLinkNet(filters[2], filters[1])
        self.decoder2 = DecoderBlockLinkNet(filters[1], filters[0])
        self.decoder1 = DecoderBlockLinkNet(filters[0], filters[0])

        # Final Classifier
        self.finaldeconv1 = nn.ConvTranspose2d(filters[0], 32, 3, stride=2)
        self.finalrelu1 = nn.ReLU(inplace=True)
        self.finalconv2 = nn.Conv2d(32, 32, 3)
        self.finalrelu2 = nn.ReLU(inplace=True)
        self.finalconv3 = nn.Conv2d(32, num_classes, 2, padding=1)

    # noinspection PyCallingNonCallable
    def forward(self, x):
        # Encoder
        x = self.firstconv(x)
        x = self.firstbn(x)
        x = self.firstrelu(x)
        x = self.firstmaxpool(x)
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)

        # Decoder with Skip Connections
        d4 = self.decoder4(e4) + e3
        d3 = self.decoder3(d4) + e2
        d2 = self.decoder2(d3) + e1
        d1 = self.decoder1(d2)

        # Final Classification
        f1 = self.finaldeconv1(d1)
        f2 = self.finalrelu1(f1)
        f3 = self.finalconv2(f2)
        f4 = self.finalrelu2(f3)
        f5 = self.finalconv3(f4)

        if self.num_classes > 1:
            x_out = F.log_softmax(f5, dim=1)
        else:
            x_out = f5
        return x_out


class Conv3BN(nn.Module):
    def __init__(self, in_: int, out: int, bn=False):
        super().__init__()
        self.conv = conv3x3(in_, out)
        self.bn = nn.BatchNorm2d(out) if bn else None
        self.activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        x = self.activation(x)
        return x


class UNetModule(nn.Module):
    def __init__(self, in_: int, out: int):
        super().__init__()
        self.l1 = Conv3BN(in_, out)
        self.l2 = Conv3BN(out, out)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        return x


class UNet(nn.Module):
    """
    Vanilla UNet.

    Implementation from https://github.com/lopuhin/mapillary-vistas-2017/blob/master/unet_models.py
    """
    output_downscaled = 1
    module = UNetModule

    def __init__(self,
                 input_channels: int = 3,
                 filters_base: int = 32,
                 down_filter_factors=(1, 2, 4, 8, 16),
                 up_filter_factors=(1, 2, 4, 8, 16),
                 bottom_s=4,
                 num_classes=1,
                 add_output=True):
        super().__init__()
        self.num_classes = num_classes
        assert len(down_filter_factors) == len(up_filter_factors)
        assert down_filter_factors[-1] == up_filter_factors[-1]
        down_filter_sizes = [filters_base * s for s in down_filter_factors]
        up_filter_sizes = [filters_base * s for s in up_filter_factors]
        self.down, self.up = nn.ModuleList(), nn.ModuleList()
        self.down.append(self.module(input_channels, down_filter_sizes[0]))
        for prev_i, nf in enumerate(down_filter_sizes[1:]):
            self.down.append(self.module(down_filter_sizes[prev_i], nf))
        for prev_i, nf in enumerate(up_filter_sizes[1:]):
            self.up.append(self.module(
                down_filter_sizes[prev_i] + nf, up_filter_sizes[prev_i]))
        pool = nn.MaxPool2d(2, 2)
        pool_bottom = nn.MaxPool2d(bottom_s, bottom_s)
        upsample = nn.Upsample(scale_factor=2)
        upsample_bottom = nn.Upsample(scale_factor=bottom_s)
        self.downsamplers = [None] + [pool] * (len(self.down) - 1)
        self.downsamplers[-1] = pool_bottom
        self.upsamplers = [upsample] * len(self.up)
        self.upsamplers[-1] = upsample_bottom
        self.add_output = add_output
        if add_output:
            self.conv_final = nn.Conv2d(up_filter_sizes[0], num_classes, 1)

    def forward(self, x):
        xs = []
        for downsample, down in zip(self.downsamplers, self.down):
            x_in = x if downsample is None else downsample(xs[-1])
            x_out = down(x_in)
            xs.append(x_out)

        x_out = xs[-1]
        for x_skip, upsample, up in reversed(
                list(zip(xs[:-1], self.upsamplers, self.up))):
            x_out = upsample(x_out)
            x_out = up(torch.cat([x_out, x_skip], 1))

        if self.add_output:
            x_out = self.conv_final(x_out)
            if self.num_classes > 1:
                x_out = F.log_softmax(x_out, dim=1)
        return x_out


class AlbuNet(nn.Module):
    """
        UNet (https://arxiv.org/abs/1505.04597) with Resnet34(https://arxiv.org/abs/1512.03385) encoder
        Proposed by Alexander Buslaev: https://www.linkedin.com/in/al-buslaev/
        """

    def __init__(self, num_classes=1, num_filters=32, pretrained=False, is_deconv=False):
        """
        :param num_classes:
        :param num_filters:
        :param pretrained:
            False - no pre-trained network is used
            True  - encoder is pre-trained with resnet34
        :is_deconv:
            False: bilinear interpolation is used in decoder
            True: deconvolution is used in decoder
        """
        super().__init__()
        self.num_classes = num_classes

        self.pool = nn.MaxPool2d(2, 2)

        self.encoder = torchvision.models.resnet34(pretrained=pretrained)

        self.relu = nn.ReLU(inplace=True)

        self.conv1 = nn.Sequential(self.encoder.conv1,
                                   self.encoder.bn1,
                                   self.encoder.relu,
                                   self.pool)

        self.conv2 = self.encoder.layer1

        self.conv3 = self.encoder.layer2

        self.conv4 = self.encoder.layer3

        self.conv5 = self.encoder.layer4

        self.center = DecoderBlock(512, num_filters * 8 * 2, num_filters * 8, is_deconv)

        self.dec5 = DecoderBlock(512 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv)
        self.dec4 = DecoderBlock(256 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv)
        self.dec3 = DecoderBlock(128 + num_filters * 8, num_filters * 4 * 2, num_filters * 2, is_deconv)
        self.dec2 = DecoderBlock(64 + num_filters * 2, num_filters * 2 * 2, num_filters * 2 * 2, is_deconv)
        self.dec1 = DecoderBlock(num_filters * 2 * 2, num_filters * 2 * 2, num_filters, is_deconv)
        self.dec0 = ConvRelu(num_filters, num_filters)
        self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv2 = self.conv2(conv1)
        conv3 = self.conv3(conv2)
        conv4 = self.conv4(conv3)
        conv5 = self.conv5(conv4)

        center = self.center(self.pool(conv5))

        dec5 = self.dec5(torch.cat([center, conv5], 1))

        dec4 = self.dec4(torch.cat([dec5, conv4], 1))
        dec3 = self.dec3(torch.cat([dec4, conv3], 1))
        dec2 = self.dec2(torch.cat([dec3, conv2], 1))
        dec1 = self.dec1(dec2)
        dec0 = self.dec0(dec1)

        if self.num_classes > 1:
            x_out = F.log_softmax(self.final(dec0), dim=1)
        else:
            x_out = self.final(dec0)

        return x_out


In [60]:

import torch
from torch.autograd import Variable
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True
# from unet_models import UNet16, unet11
class FaceSegGPU:
    def __init__(self, bs, size=256):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.net = unet11('unet_celeba.pth', pretrained=True).to(self.device)

        # self.net = UNet16(pretrained=True).to(self.device)
        # self.net.load_state_dict(torch.load('unet16.pth'))
        self.net.eval()
        sample = Variable(torch.rand(bs,3,size,size).to(self.device))
        self.net(sample)
        #  = torch.jit.trace(self.net, sample)
        print('___init___')

    def get_mask(self, images, shape):
        # images = Variable(torch.tensor(images, dtype=torch.float,requires_grad=False).to(device=self.device))
        pred = self.net(images)
        pred= torch.nn.functional.interpolate(pred, size=[shape[1], shape[2]])
        pred = pred.squeeze()
        mask = (pred > 0.8)
        segmentation = mask.cpu().numpy()
        return segmentation.astype('float')

    def apply_masks(self, frames_transformed, frames):
        masks = self.get_mask(frames_transformed, frames.shape)
        frames[masks==0] = 0.0
        return frames


In [61]:
import numpy as np
from scipy.signal import medfilt, decimate
import cv2
from torchvision import transforms
import pdb
from PIL import Image
import time
import torch
from torch.autograd import Variable

def scale_pulse(p):
    p = p - np.min(p)
    p = p/np.max(p)
    p-=0.5
    p*=2
    return p

def moving_avg(signal, w_s):
    ones = np.ones(w_s) / w_s
    moving_avg = np.convolve(signal, ones, 'valid')
    return moving_avg

def compute_snr(coef):
    num_of_bins = len(coef)
    max_bin_index = np.argmax(coef)
    signal_bins = coef[max_bin_index] + (coef[max_bin_index * 2] if max_bin_index * 2 < num_of_bins else 0)
    noise_bins = np.sum(coef) - signal_bins
    snr = 20*(np.log10(signal_bins/noise_bins))
    return snr

def post_process(values, w_s, k_s):
    ones = np.ones(w_s) / w_s
    moving_avg = np.convolve(values, ones, 'valid')
    decimated = decimate(moving_avg, w_s)
    filterd =  medfilt(decimated, k_s)
    return filterd

def compute_mean(frames):
    mmm = np.true_divide(frames.sum(axis=(1,2)),(frames!=0).sum(axis=(1,2)))
    return mmm

def transform_frames(frames, device, size=256):

    frames_copy = np.copy(frames)

    frames_transposed = np.transpose(frames_copy, (0,3,1,2))

    mean = torch.tensor(np.array([0.485, 0.456, 0.406]), dtype=torch.float).to(device=device)
    std = torch.tensor(np.array([0.229, 0.224, 0.225]), dtype=torch.float).to(device=device)

    tensors = Variable(torch.tensor(frames_transposed, dtype=torch.float).to(device=device)).div(255)

    resized = torch.nn.functional.interpolate(tensors, size=(size, size))

    normalized = (resized - mean[None, :, None, None]) / std[None, :, None, None]


    return normalized




def get_transform(size=256):
    t = transforms.Compose([
        transforms.Resize(size=(size, size)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    return t

def transform_single_frame(frames, size=256):
    f = get_transform()
    shape = frames.shape

    tranformed_frames = np.zeros((shape[0], 3,  shape[1], shape[2]))

    for i in range(shape[0]):
        tranformed_frames[i] = f(Image.fromarray(frames[i]))
    return tranformed_frames


In [62]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 18 11:15:49 2019

@author: nasir

Implements
Wavelet for post processing of pulse signal

"""
from scipy.signal import convolve2d
import numpy as np
import math
# import pywt
from scipy.signal import convolve2d
import pycwt as wavelet

class Wavelet():
    def __init__(self, sr):
        self.samplingRate = sr
        self.minFreq = 0.75 #
        self.maxFreq = 3 #
#
    def get_scales(self):
        MorletFourierFactor = 4* math.pi / (6+math.sqrt(2+ 6**2))
        """take default smallest scale (corresponding to Nyquist frequency)"""
        scMin = 1 / (0.5 * self.samplingRate * MorletFourierFactor)
        """32 scales per octave"""
        ds = 1/32
        """  we do not consider low freq components """
        scMax = 1/(0.5 * self.minFreq * MorletFourierFactor)
        nSc =  math.floor(math.log2(scMax/scMin) /ds)

        scales = scMin * (2 ** (np.arange(0, nSc+1) * ds))
        return scales

    def get_cwt(self, signal):
        scales = self.get_scales()

#        """get frequencies range of interest 0.67 ~~ 4 Hz"""
        MorletFourierFactor = 4* math.pi / (6+math.sqrt(2+ 6**2))
        freqs = 1 / (scales * MorletFourierFactor)

        coef, scales, _, coi, fft, fftfreqs = wavelet.cwt(signal, 1/self.samplingRate, wavelet='morlet', freqs=freqs)


        firstScaleIndex = np.where(freqs < self.maxFreq)[0][0]
        lastScaleIndex = np.where(freqs > self.minFreq)[0][-1]

        energyProfile = np.abs(coef)

        max_index = np.argmax(energyProfile[firstScaleIndex:lastScaleIndex,:], axis=0)
        instantPulseRate = 60 * freqs[firstScaleIndex + max_index]

        return coef, instantPulseRate

    def get_instant_beats(self, energyProfile):
        scales = self.get_scales()
        MorletFourierFactor = 4* math.pi / (6+math.sqrt(2+ 6**2))
        freqs = 1 / (scales * MorletFourierFactor)
        firstScaleIndex = np.where(freqs < self.maxFreq)[0][0]
        lastScaleIndex = np.where(freqs < self.minFreq)[0][0]
        max_index = np.argmax(energyProfile[firstScaleIndex:lastScaleIndex,:], axis=0)
        instantPulseRate = 60 * freqs[firstScaleIndex + max_index - 1]
        return instantPulseRate


In [63]:
import numpy as np
from cdf import CDF
from asf import ASF
from numpy.linalg import inv

from scipy.fftpack import rfftfreq, rfft
import cv2
from torchvision import transforms
import pdb
from PIL import Image

PRE_STEP_ASF = False
PRE_STEP_CDF = False

class Pulse():
    def __init__(self, framerate, signal_size, batch_size, image_size=256):
        self.framerate = float(framerate)
        self.signal_size = signal_size
        self.batch_size = batch_size
        self.minFreq = 0.9 #
        self.maxFreq = 3 #
        self.fft_spec = []

    def get_pulse(self, mean_rgb):
        seg_t = 3.2
        l = int(self.framerate * seg_t)
        H = np.zeros(self.signal_size)

        B = [int(0.8 // (self.framerate / l)), int(4 // (self.framerate / l))]

        for t in range(0, (self.signal_size - l + 1)):
            # pre processing steps
            C = mean_rgb[t:t+l,:].T

            if PRE_STEP_CDF:
                C = CDF(C, B)

            if PRE_STEP_ASF:
                C = ASF(C)

            # POS
            mean_color = np.mean(C, axis=1)
            diag_mean_color = np.diag(mean_color)
            diag_mean_color_inv = np.linalg.inv(diag_mean_color)
            Cn = np.matmul(diag_mean_color_inv,C)
            projection_matrix = np.array([[0,1,-1],[-2,1,1]])
            S = np.matmul(projection_matrix,Cn)
            std = np.array([1,np.std(S[0,:])/np.std(S[1,:])])
            P = np.matmul(std,S)
            H[t:t+l] = H[t:t+l] +  (P-np.mean(P))
        return H

    def get_rfft_hr(self, signal):
        signal_size = len(signal)
        signal = signal.flatten()
        fft_data = np.fft.rfft(signal) # FFT
        fft_data = np.abs(fft_data)

        freq = np.fft.rfftfreq(signal_size, 1./self.framerate) # Frequency data

        inds= np.where((freq < self.minFreq) | (freq > self.maxFreq) )[0]
        fft_data[inds] = 0
        bps_freq=60.0*freq
        max_index = np.argmax(fft_data)
        fft_data[max_index] = fft_data[max_index]**2
        self.fft_spec.append(fft_data)
        HR =  bps_freq[max_index]
        return HR


In [64]:
import cv2
import numpy as np
from pulse import Pulse
import time
from threading import Lock, Thread
from plot_cont import DynamicPlot
from capture_frames import CaptureFrames
import pandas as pd
from matplotlib import pyplot as plt

from utils import *
import multiprocessing as mp
import sys


class ProcessMasks():

    def __init__(self, sz=270, fs=30, bs=30, size=256):
        print('init')
        self.stop = False
        self.masked_batches = []
        self.batch_mean = []
        self.signal_size = sz
        self.batch_size = bs
        self.signal = np.zeros((sz, 3))
        self.pulse = Pulse(fs, sz, bs, size)
        self.hrs = []
        self.save_results = True

    def __call__(self, pipe, plot_pipe, source):
        self.pipe = pipe
        self.plot_pipe = plot_pipe
        self.source = source
        compute_mean_thread =  Thread(target=self.compute_mean)
        compute_mean_thread.start()

        extract_signal_thread =  Thread(target=self.extract_signal)
        extract_signal_thread.start()

        self.rec_frames()

        compute_mean_thread.join()
        extract_signal_thread.join()

    def rec_frames(self):
        while True and not self.stop:
            data = self.pipe.recv()

            if data is None:
                self.terminate()
                break
            batch = data[0]

            self.masked_batches.append(batch)

    def process_signal(self, batch_mean):
        size = self.signal.shape[0]
        b_size = batch_mean.shape[0]

        self.signal[0:size-b_size] = self.signal[b_size:size]
        self.signal[size-b_size:] = batch_mean
        p = self.pulse.get_pulse(self.signal)
        p = moving_avg(p, 6)
        hr = self.pulse.get_rfft_hr(p)
        if len(self.hrs) > 300: self.hrs.pop(0)

        self.hrs.append(hr)
        if self.plot_pipe is not None and self.stop:
            self.plot_pipe.send(None)
        elif self.plot_pipe is not None:
            self.plot_pipe.send([p, self.hrs])
        else:
            hr_fft = moving_avg(self.hrs, 3)[-1] if len(self.hrs) > 5 else self.hrs[-1]
            sys.stdout.write(f'\rHr: {round(hr_fft, 0)}')
            sys.stdout.flush()

    def extract_signal(self):
        signal_extracted = 0

        while True and not self.stop:
            if len(self.batch_mean) == 0:
                time.sleep(0.01)
                continue

            mean_dict = self.batch_mean.pop(0)
            mean = mean_dict['mean']

            if mean_dict['face_detected'] == False:
                if self.plot_pipe is not None:
                    self.plot_pipe.send('no face detected')
                continue
            if signal_extracted >= self.signal_size:
                self.process_signal(mean)
            else:
                self.signal[signal_extracted: signal_extracted + mean.shape[0]] = mean
            signal_extracted+=mean.shape[0]


    def compute_mean(self):
        curr_batch_size = 0
        batch = None
        while True and not self.stop:
            if len(self.masked_batches) == 0:
                time.sleep(0.01)
                continue

            mask = self.masked_batches.pop(0)
            if batch is None:
                batch = np.zeros((self.batch_size, mask.shape[0], mask.shape[1], mask.shape[2]))

            if curr_batch_size < (self.batch_size - 1):
                batch[curr_batch_size] = mask
                curr_batch_size+=1
                continue

            batch[curr_batch_size] = mask
            curr_batch_size = 0

            non_zero_pixels = (batch!=0).sum(axis=(1,2))
            total_pixels = batch.shape[1] * batch.shape[2]
            avg_skin_pixels = non_zero_pixels.mean()
            m = {'face_detected': True, 'mean': np.zeros((self.batch_size, 3))}
            if (avg_skin_pixels + 1) / (total_pixels) < 0.05:
                m['face_detected'] = False
            else:
                m['mean'] = np.true_divide(batch.sum(axis=(1,2)), non_zero_pixels+1e-6)

            self.batch_mean.append(m)

    def terminate(self):

        if self.plot_pipe is not None:
            self.plot_pipe.send(None)
        self.savePlot(self.source)
        self.saveresults()
        self.stop = True

    def saveresults(self):
        """
        saves numpy array of heart rates as hrs
        saves numpy array of power spectrum as fft_spec
        """
        np.save('hrs', np.array(self.hrs))
        np.save('fft_spec', np.array(self.pulse.fft_spec))

    def savePlot(self, path):
        if self.save_results == False:
            return

        # path = path.replace   ('/media/munawar/','/munawar-desktop/')
        # fig_path = path[40:].replace("/","_")

        # file_path = path.replace('video.avi','gt_HR.csv')
        # gt_HR = pd.read_csv(file_path, index_col=False).values
        if len(self.hrs) == 0:
            return

        ax1 = plt.subplot(1,1,1)
        ax1.set_title('HR')
        ax1.set_ylim([20, 180])
        ax1.plot(moving_avg(self.hrs, 6))

        # ax3 = plt.subplot(1,2,2)
        # ax3.set_title('GT')
        # ax3.set_ylim([20, 180])
        # ax3.plot(gt_HR[8:])

        plt.tight_layout()
        plt.savefig(f'results.png')
        plt.close()






In [65]:

import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from utils import *
from scipy.signal import medfilt, decimate

plt.ion()
class DynamicPlot():
    def __init__(self, signal_size, bs):
        self.batch_size = bs
        self.signal_size = signal_size
        self.launched = False

    def launch_fig(self):
        self.fig, (self.pulse_ax, self.hr_axis)= plt.subplots(2, 1)

        self.pulse_to_plot = np.zeros(self.signal_size)
        self.hrs_to_plot = np.zeros(self.signal_size)
        # self.rgb_to_plot = np.zeros(self.signal_size)

        self.hr_texts = self.pulse_ax.text(0.1, 0.9,'0', ha='center', va='center', transform=self.pulse_ax.transAxes)
        self.pulse_ax.set_title('BVP')
        self.hr_axis.set_title('Heart Rate')
        # self.rgb_axis.set_title('color')
        self.pulse_ax.set_autoscaley_on(True)

        self.pulse_ax.plot(self.pulse_to_plot)
        self.hr_axis.plot(self.hrs_to_plot)
        # self.rgb_axis.plot(self.rgb_to_plot)

        self.pulse_ax.set_ylim(-3,3)
        self.hr_axis.set_ylim(0,180)
        self.launched = True

        plt.tight_layout()
        plt.show()

    def __call__(self, pipe):
        if self.launched == False: self.launch_fig()
        self.pipe = pipe
        self.call_back()

    def call_back(self):
        while True:
            data = self.pipe.recv()
            if data is None:
                self.terminate()
                break
            elif data == 'no face detected':
                self.update_no_face()
            else:
                self.update_data(data[0], data[1])

    def update_no_face(self):
        hr_text = 'HR: NaN'
        self.hr_texts.set_text(hr_text)

        scaled = np.zeros(10)
        for i in range(0, len(scaled)):
            self.pulse_to_plot[0:self.signal_size-1] = self.pulse_to_plot[1:]
            self.pulse_to_plot[-1] = scaled[i]
            self.update_plot(self.pulse_ax, self.pulse_to_plot)

            self.hrs_to_plot[0:self.signal_size-1] = self.hrs_to_plot[1:]
            self.hrs_to_plot[-1] = 0
            self.update_plot(self.hr_axis, self.hrs_to_plot)
            self.re_draw()

    def update_data(self, p, hrs):

        hr_fft = moving_avg(hrs, 3)[-1] if len(hrs) > 5 else hrs[-1]
        hr_text = 'HR: ' + str(int(hr_fft))
        self.hr_texts.set_text(hr_text)

        # ma = moving_avg(p[-self.batch_size:], 6)
        batch = p[-self.batch_size:]
        decimated_p = decimate(batch, 3)
        # filterd_p =  medfilt(decimated_p, 5)
        scaled = scale_pulse(decimated_p)
        for i in range(0, len(scaled)):
            self.pulse_to_plot[0:self.signal_size-1] = self.pulse_to_plot[1:]
            self.pulse_to_plot[-1] = scaled[i]
            self.update_plot(self.pulse_ax, self.pulse_to_plot)
            self.hrs_to_plot[0:self.signal_size-1] = self.hrs_to_plot[1:]
            self.hrs_to_plot[-1] = hr_fft
            self.update_plot(self.hr_axis, self.hrs_to_plot)
            self.re_draw()


    def update_plot(self, axis, y_values):
        line = axis.lines[0]
        line.set_xdata(np.arange(len(y_values)))
        line.set_ydata(y_values)
        axis.relim()
        axis.autoscale_view()
    def re_draw(self):
        #print(self.fig.canvas.tostring_rgb())
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()


    def terminate(self):
        """
        saves numpy array of rPPG signal as pulse
        """
        np.save('pulse', self.pulse_to_plot)
        plt.close('all')





In [66]:

import cv2
import numpy as np
from pulse import Pulse
import time
from threading import Lock, Thread
from plot_cont import DynamicPlot
from capture_frames import CaptureFrames
from process_mask import ProcessMasks

from utils import *
import multiprocessing as mp
import sys
from optparse import OptionParser

class RunPOS():
    def __init__(self,  sz=270, fs=28, bs=30, plot=False):
        self.batch_size = bs
        self.frame_rate = fs
        self.signal_size = sz
        self.plot = plot

    def __call__(self, source):
        time1=time.time()

        mask_process_pipe, chil_process_pipe = mp.Pipe()
        self.plot_pipe = None
        if self.plot:
            self.plot_pipe, plotter_pipe = mp.Pipe()
            self.plotter = DynamicPlot(self.signal_size, self.batch_size)
            self.plot_process = mp.Process(target=self.plotter, args=(plotter_pipe,), daemon=True)
            self.plot_process.start()

        process_mask = ProcessMasks(self.signal_size, self.frame_rate, self.batch_size)

        mask_processer = mp.Process(target=process_mask, args=(chil_process_pipe, self.plot_pipe, source, ), daemon=True)
        mask_processer.start()

        capture = CaptureFrames(self.batch_size, source, show_mask=True)
        capture(mask_process_pipe, source)


        mask_processer.join()
        if self.plot:
            self.plot_process.join()
        time2=time.time()
        time2=time.time()
        print(f'time {time2-time1}')

def get_args():
    parser = OptionParser()
    parser.add_option('-s', '--source', dest='source', default=0,
                        help='Signal Source: 0 for webcam or file path')
    parser.add_option('-b', '--batch-size', dest='batchsize', default=30,
                        type='int', help='batch size')
    parser.add_option('-f', '--frame-rate', dest='framerate', default=25,
                        help='Frame Rate')

    (options, _) = parser.parse_args()
    return options

if __name__=="__main__":
    args = get_args()
    source = args.source
    runPOS = RunPOS(270, args.framerate, args.batchsize, True)
    runPOS(source)


init


ValueError: could not convert string to float: 'C:\\Users\\lenovo\\AppData\\Roaming\\jupyter\\runtime\\kernel-13767828-9d5f-4c1a-a240-a302f5a8367d.json'